In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat

In [ ]:
# archivo de Matlab con los datos.
mat = loadmat("data/servers.mat")
X = mat["X"]

### Deplegamos la data de Servidores

In [ ]:
plt.figure(figsize=(20,10))
plt.scatter(X[:,0],X[:,1],marker="x", c = 'green')
plt.xlim(0,30)
plt.ylim(0,30)
plt.xlabel("Latencia (ms)")
plt.ylabel(" Transferencia Datos (mb/s)")
plt.show()

### Calculo de parametros Gaussianos

Dada una variable x, vamos a estimar la media y la desviacion estandard.

$p(x;\mu,\sigma^2) = \frac{1}{\sqrt{2\pi\sigma^2}}e^{\frac{(x-\mu)^2}{2\sigma^2}}$

$\mu_i = \frac{1}{m}\sum^m_{j=1}x^{(j)}$

$\sigma^2_i = \frac{1}{m}\sum^m_{j=1}(x^{(j)} - \mu_j)^2$

In [ ]:
def estimarParametrosGaussianos(X):
    
    # cantidad de datos
    m = X.shape[0]
    
    # se calcula la media
    sum_ = np.sum(X,axis=0)
    mu = 1/m *sum_
    
    # se calcula la varianza
    var = 1/m * np.sum((X - mu)**2,axis=0)
    
    return mu,var

# Estimacion de ejemplo de parametros para latencia...
print("Parametros para Latencia:", estimarParametrosGaussianos(X[:,0]))
# Todos los parametros
print("Todos los Parametros:", estimarParametrosGaussianos(X))

### Estimacion de la Distribucion Gaussiana con Muliples Variables

$p(x;\mu,\Sigma) = \frac{1}{(2\pi)^{n/2}|\Sigma|^{1/2}} exp(-\frac{1}{2}(x-\mu)^T\Sigma^{-1}(x-\mu))$

In [ ]:
def distribucionGaussiana(X, mu, sigma):
    
    # cantidad de parametros mu (media)
    k = len(mu)
    
    sigma=np.diag(sigma)
    X = X - mu.T
    
    # calculo de la distribucion
    d = 1/((2*np.pi)**(k/2)*(np.linalg.det(sigma)**0.5))* np.exp(-0.5* np.sum(X @ np.linalg.pinv(sigma) * X,axis=1))
    
    return d


mu,sig = estimarParametrosGaussianos(X)
dist = distribucionGaussiana(X,mu,sig)
print("Distribucion Multivariada de X:", dist[:10],"...")
print(dist.shape)

### Escoger el Threshold e

In [ ]:
plt.figure(figsize=(20,10))
plt.hist(dist, bins = 100)
plt.xticks(np.arange(0, 0.09, 0.001))
plt.xticks(rotation=90)
plt.axvline(x=np.mean(dist), color='r', linestyle='dashed', linewidth=3)
plt.axvline(x=np.mean(dist) - 2 *  np.std(dist), color='orange', linestyle='dashed', linewidth=1)
plt.axvline(x=np.mean(dist) - 2.5 *  np.std(dist), color='green', linestyle='dashed', linewidth=1)
plt.show()

### Visualizar Anomalias

In [ ]:
plt.figure(figsize=(20,10))
plt.scatter(X[:,0],X[:,1],marker="x", c = 'green')
plt.xlim(0,30)
plt.ylim(0,30)
plt.xlabel("Latencia (ms)")
plt.ylabel(" Transferencia Datos (mb/s)")

# Threshold...
e = 0.001

# anomalias (condicion)
outliers = np.nonzero(dist<e)[0]
plt.scatter(X[outliers,0],X[outliers,1],marker ="o",facecolor="none",edgecolor="b",s=120)

plt.show()